# 自作yolov8でロゴ部分のみ抽出 version (Nishika 2nd solutionのconvnextの，AugLy部分を排除して利用)
ref: https://github.com/anyai-28/nishika_jpo_2nd_solution?tab=readme-ov-file

手順
1. テーブルデータとイメージデータの前処理(済)   
my_train_w_group_label.csv, crop_cite_images.zip, crop_test_images.zip, crop_apply_images.zip
2. 前処理したデータを移動   
3. ディレクトリ構造の再構築   
Nishika 2nd Place Solution /settings.jsonの通りにする
4. python3.8にダウングレードしてライブラリインストール
5. train_convで学習

train_conv.py -> ~train_swin.py~ -> submission.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## download directory structure and csv files

In [ ]:
%cd /content/

/content


In [ ]:
# it is the Nishika trademark competition datasets (https://competition.nishika.com/competitions/patent/data)
!pip install gdown
!gdown --folder https://drive.google.com/drive/folders/1KUpQ6ZyIawgALEsyzWBt4fr7s8TT62vM?usp=drive_link

Retrieving folder list
Processing file 1-ObndOWm0jpljV7PTps6mOqtW-2lNM77 cite.csv
Processing file 1-OLYdb0NPMHgtyCmQ93jUGp8CgzNN03a cpt-jpo-tutorial.zip
Processing file 1vYJZ-s8ppZtT66iTyJh1vSbmHEtGThCy my_train_w_group_label.csv
Processing file 1-kL8bZKY3ZQeRIIBJSd0NNq55NscIxUc sample_submission.csv
Processing file 1-cLynCOTU1QMuRSphsiOhlgIC45espef test.csv
Processing file 1-Z_rzDGZqZ-My2BOz0aUv5tjpnuLKHE6 train.csv
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1-ObndOWm0jpljV7PTps6mOqtW-2lNM77
To: /content/common/cite.csv
100% 29.6M/29.6M [00:00<00:00, 123MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-OLYdb0NPMHgtyCmQ93jUGp8CgzNN03a
To: /content/common/cpt-jpo-tutorial.zip
100% 1.02M/1.02M [00:00<00:00, 81.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vYJZ-s8ppZtT66iTyJh1vSbmHEtGThCy
To: /content/common/my_train_w_group_label.csv
100% 196k/196k [00:00<

In [ ]:
!cp /content/common/cpt-jpo-tutorial.zip /content
!unzip /content/cpt-jpo-tutorial.zip

Archive:  /content/cpt-jpo-tutorial.zip
  inflating: requirements.txt        
  inflating: __MACOSX/._requirements.txt  
   creating: data/
  inflating: data/.DS_Store          
  inflating: __MACOSX/data/._.DS_Store  
   creating: data/input/
   creating: data/output/
  inflating: data/input/.gitkeep     
  inflating: data/output/.gitkeep    
   creating: models/
  inflating: models/.gitkeep         
   creating: notebooks/
  inflating: notebooks/.DS_Store     
  inflating: __MACOSX/notebooks/._.DS_Store  
  inflating: notebooks/tutorial_check_train_data.ipynb  
  inflating: __MACOSX/notebooks/._tutorial_check_train_data.ipynb  
   creating: scripts/
  inflating: scripts/03_tutorial_infer.py  
  inflating: __MACOSX/scripts/._03_tutorial_infer.py  
  inflating: scripts/01_tutorial_preprocess.py  
  inflating: __MACOSX/scripts/._01_tutorial_preprocess.py  
  inflating: scripts/.DS_Store       
  inflating: __MACOSX/scripts/._.DS_Store  
  inflating: scripts/02_tutorial_train.py  
  infl

In [ ]:
!cp -r /content/common/* /content/data/input

## download yolov8 Croped input image

crop_cite_images is **too large** cannot download gdown
- you can use the way
ref: https://yuiga.dev/blog/posts/%E5%B7%A8%E5%A4%A7%E3%83%87%E3%83%BC%E3%82%BFgdown%E3%81%AEaccess_denied%E5%95%8F%E9%A1%8C%E3%81%AF%E8%AB%A6%E3%82%81%E3%82%88%E3%81%86/


>解決策
>1. 当該URLからファイルIDを抜き出す (URLのどこがidなのかは適当にググってください)
>2. OAuth 2.0 Playgroundにアクセス
>3. “Drive API v3”を探してクリック→ 「 https://www.googleapis.com/auth/drive.>readonly 」と書いてある所をクリック
>4. “exchange authorization code for tokens"をクリックして"Access Token"を取得
>5. curl -H "Authorization: Bearer <access-token>" https://www.googleapis.com/drive/v3/files/<file-id>?alt=media -o <output-path> を実行
>6. ```<access-token>, <file-id>, <output-path>```はご自身に適切なものを入れてください

- or you can dowonload it (https://drive.google.com/drive/folders/1-1fqMU3DXTR-IAR7mRS-biMZPD6bSrE0?usp=drive_link)
and upload your colab.


In [ ]:
# FILE_ID='1-ASA9GDzzk76xplGxsIRZzjqJwdNwlho'
# FILE_NAME='crop_cite_images.zip'

# yolov8 crop_cite_images https://drive.google.com/file/d/1-ASA9GDzzk76xplGxsIRZzjqJwdNwlho/view?usp=drive_link
# !curl -H "Authorization: Bearer <access token>" https://www.googleapis.com/drive/v3/files/1-ASA9GDzzk76xplGxsIRZzjqJwdNwlho?alt=media -o crop_cite_images.zip
!curl -H "Authorization: Bearer " https://www.googleapis.com/drive/v3/files/1-ASA9GDzzk76xplGxsIRZzjqJwdNwlho?alt=media -o crop_cite_images.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 37.3G  100 37.3G    0     0  95.6M      0  0:06:39  0:06:39 --:--:-- 73.3M


In [ ]:
# !curl -H "Authorization: Bearer <acccess token>" https://www.googleapis.com/drive/v3/files/1-7wwn4hiKXZ3hpU1j7yyKlO7mlv43XbV?alt=media -o crop_test_images.zip
# !curl -H "Authorization: Bearer <acccess token>" https://www.googleapis.com/drive/v3/files/1DLVYkwBHUu3y7r5SKZEihfS7jHq6qApq?alt=media -o crop_train_apply_images.zip
# !curl -H "Authorization: Bearer <acccess token>" https://www.googleapis.com/drive/v3/files/1-28QPaUHLJZlbWKTUK4RMJvV8TenVlj8?alt=media -o crop_train_cite_images.zip


# yolov8 crop_test_images https://drive.google.com/file/d/1-7wwn4hiKXZ3hpU1j7yyKlO7mlv43XbV/view?usp=drive_link
!curl -H "Authorization: Bearer " https://www.googleapis.com/drive/v3/files/1-7wwn4hiKXZ3hpU1j7yyKlO7mlv43XbV?alt=media -o crop_test_images.zip
# yolov8 crop_train_apply_images https://drive.google.com/file/d/1DLVYkwBHUu3y7r5SKZEihfS7jHq6qApq/view?usp=drive_link
!curl -H "Authorization: Bearer " https://www.googleapis.com/drive/v3/files/1DLVYkwBHUu3y7r5SKZEihfS7jHq6qApq?alt=media -o crop_train_apply_images.zip
# yolov8 crop_train_cite_images https://drive.google.com/file/d/1-28QPaUHLJZlbWKTUK4RMJvV8TenVlj8/view?usp=drive_link
!curl -H "Authorization: Bearer " https://www.googleapis.com/drive/v3/files/1-28QPaUHLJZlbWKTUK4RMJvV8TenVlj8?alt=media -o crop_train_cite_images.zip

# !gdown https://drive.google.com/uc?id=1-7wwn4hiKXZ3hpU1j7yyKlO7mlv43XbV  # yolov8 crop_test_images https://drive.google.com/file/d/1-7wwn4hiKXZ3hpU1j7yyKlO7mlv43XbV/view?usp=drive_link
# !gdown https://drive.google.com/uc?id=1DLVYkwBHUu3y7r5SKZEihfS7jHq6qApq  # yolov8 crop_train_apply_images https://drive.google.com/file/d/1DLVYkwBHUu3y7r5SKZEihfS7jHq6qApq/view?usp=drive_link
# !gdown https://drive.google.com/uc?id=1-28QPaUHLJZlbWKTUK4RMJvV8TenVlj8  # yolov8 crop_train_cite_images https://drive.google.com/file/d/1-28QPaUHLJZlbWKTUK4RMJvV8TenVlj8/view?usp=drive_link

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 71.1M  100 71.1M    0     0  4441k      0  0:00:16  0:00:16 --:--:-- 17.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  111M  100  111M    0     0  33.0M      0  0:00:03  0:00:03 --:--:-- 33.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 95.6M  100 95.6M    0     0  4288k      0  0:00:22  0:00:22 --:--:-- 26.5M


In [ ]:
!unzip /content/crop_cite_images.zip -d /content/data/input/
!unzip /content/crop_train_apply_images.zip -d /content/data/input/
!unzip /content/crop_train_cite_images.zip -d /content/data/input/
!unzip /content/crop_test_images.zip -d /content/data/input/

In [ ]:
!rm /content/crop_cite_images.zip
!rm /content/crop_train_apply_images.zip
!rm /content/crop_train_cite_images.zip
!rm /content/crop_test_images.zip

## jsonの通りのディレクトリ構成にしておく
ref: https://github.com/anyai-28/nishika_jpo_2nd_solution

In [ ]:
%cd /content/

/content


In [ ]:
"""
{
    "RAW_DATA_DIR": "./data/",
    "PROCESSED_DATA_DIR": "./data/processed/",
    "MODEL_CHECKPOINT_DIR": "./models/",
    "PRETRAINED_MODEL_CHECKPOINT_DIR": "./pretrained_models/",
    "USE_PRETRAIN": "False",
    "LOGS_DIR": "./logs/",
    "SUBMISSION_DEVICE": "cuda",
    "SUBMISSION_DIR": "./submissions/"}
"""
!mkdir ./data/processed
!mkdir ./pretrained_models/
!mkdir ./submissions/

!mkdir ./config
# !mkdir ./pretrained_models/swin
!mkdir ./pretrained_models/convnext
!mkdir ./logs
# !mkdir ./logs/swin
!mkdir ./logs/convnext
!mkdir ./models/convnext

import my_train_w_group_label.csv, config.yaml and settings.json

In [ ]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1ZOmSdpwuiPvy8PQ0QBf2NaCJ6yh01WE2 # my preprocessed csv
!mv /content/my_train_w_group_label.csv /content/data/input/train_w_group_label.csv

Downloading...
From: https://drive.google.com/uc?id=1ZOmSdpwuiPvy8PQ0QBf2NaCJ6yh01WE2
To: /content/my_train_w_group_label.csv
100% 196k/196k [00:00<00:00, 108MB/s]


In [ ]:
!gdown https://drive.google.com/uc?id=12F5BWiQdcnS1-FvHRMkTuLFa7lxoPUOu #config_train_convnext.yaml https://drive.google.com/file/d/12F5BWiQdcnS1-FvHRMkTuLFa7lxoPUOu/view?usp=drive_link
!mv /content/config_train_convnext.yaml /content/config

Downloading...
From: https://drive.google.com/uc?id=12F5BWiQdcnS1-FvHRMkTuLFa7lxoPUOu
To: /content/config_train_convnext.yaml
100% 6.02k/6.02k [00:00<00:00, 25.9MB/s]


In [ ]:
!gdown https://drive.google.com/uc?id=12QkWUsjk_JOLT1xI5kWYifcw3nWkGTpn #settings.json https://drive.google.com/file/d/12QkWUsjk_JOLT1xI5kWYifcw3nWkGTpn/view?usp=drive_link
# !mv /content/settings.json /content/

Downloading...
From: https://drive.google.com/uc?id=12QkWUsjk_JOLT1xI5kWYifcw3nWkGTpn
To: /content/settings.json
100% 320/320 [00:00<00:00, 1.73MB/s]


In [ ]:
!mv /content/data/input/crop_train_apply_images /content/data/processed/
!mv /content/data/input/crop_train_cite_images /content/data/processed/
!mv /content/data/input/crop_test_images /content/data/processed
!mv /content/data/input/crop_cite_images /content/data/processed/
!mv /content/data/input/train_w_group_label.csv /content/data/processed/
!mv /content/data/input/test.csv /content/data/
!mv /content/data/input/cite.csv /content/data/
!mv /content/data/input/sample_submission.csv /content/data/

In [ ]:
!mv /content/data/processed/crop_train_apply_images /content/data/processed/crop_apply_images

## Google Colabで過去のバージョンが動くようにする

python 3.8.8   
cuda 11.8

### cuda11.*系へのダウングレード

In [ ]:
!nvidia-smi

Mon Dec 18 09:42:17 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

python3.10.*   
cuda11.8

In [ ]:
#cuda 11.*系にダウングレード
#https://developer.nvidia.com/cuda-11-8-0-download-archive?target_os=Linux&target_arch=x86_64&Distribution=Ubuntu&target_version=22.04&target_type=deb_local

!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
!mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda-repo-ubuntu2204-11-8-local_11.8.0-520.61.05-1_amd64.deb
!dpkg -i cuda-repo-ubuntu2204-11-8-local_11.8.0-520.61.05-1_amd64.deb
!cp /var/cuda-repo-ubuntu2204-11-8-local/cuda-*-keyring.gpg /usr/share/keyrings/
!apt-get update
!apt-get -y install cuda-11.8

--2023-12-18 09:42:19--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu2204.pin’

cuda-ubuntu2204.pin 100%[===================>]     190  --.-KB/s    in 0s      

2023-12-18 09:42:19 (5.90 MB/s) - ‘cuda-ubuntu2204.pin’ saved [190/190]

--2023-12-18 09:42:19--  https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda-repo-ubuntu2204-11-8-local_11.8.0-520.61.05-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 O

In [ ]:
!ls -d /usr/local/cuda-*
!which nvcc

/usr/local/cuda-11  /usr/local/cuda-11.8  /usr/local/cuda-12  /usr/local/cuda-12.2
/usr/local/cuda/bin/nvcc


In [ ]:
import os
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-11.8/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-11.8/lib64:{ld}"
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


#### 環境変数設定

In [ ]:
import os
path_org = os.environ['PATH']
path_38 = '/usr/local_py38'
path_38bin = f'{path_38}/bin'
os.environ['PATH'] = f'{path_38bin}:{path_org}'

import sys
sys.path.append(f'{path_38}/lib/python3.8/site-packages')

#### conda導入

In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh | tail -n 1
!chmod +x mini.sh
!bash ./mini.sh -b -f -p $path_38 | tail -n 1

--2023-12-18 09:47:58--  https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89817099 (86M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  85.66M   246MB/s    in 0.3s    

2023-12-18 09:47:58 (246 MB/s) - ‘mini.sh’ saved [89817099/89817099]

    in Miniconda3: /usr/local_py38


In [ ]:
# 結果確認
!which conda
!which pip

/usr/local_py38/bin/conda
/usr/local_py38/bin/pip


#### ライブラリ追加導入

In [ ]:
# ref fixed 3.8.8 successed https://github.com/conda/conda/issues/10209
!conda install -q -y conda python=3.8.8 --prefix $path_38 | tail -n 1
!conda install -q -y jupyter --prefix $path_38 | tail -n 1
!conda install -q -y google-colab -c conda-forge --prefix $path_38 | tail -n 1

Executing transaction: ...working... done
Executing transaction: ...working... done
Executing transaction: ...working... done


#### カーネル追加

In [ ]:
!python -m ipykernel install --name "py38" --user
# 結果確認
!jupyter-kernelspec list

Installed kernelspec py38 in /root/.local/share/jupyter/kernels/py38
Available kernels:
  py38       /root/.local/share/jupyter/kernels/py38
  python3    /usr/local_py38/share/jupyter/kernels/python3
  ir         /usr/local/share/jupyter/kernels/ir


### you must change runtime from python3 to py38 in toolber( Runtime -> change runtime time )

#### カーネル切替え

In [ ]:
 # UI経由で実施
 # メニューから「ランタイム」「ランタイムのタイプを変更」
 # ランタイムのタイプ: 「Python 3」→「py38」 で「保存」

In [ ]:
# 新カーネルの動作テスト
1+1

2

 #### 環境変数再設定
  INFO: 原因不明（時間を空けてセルを実行したときとか）で実行停止するときがあるので，その際はrestart session してから，以下の環境変数を設定しなおしてセル再実行

In [ ]:
!ls -d /usr/local/cuda-*
!which nvcc

/usr/local/cuda-11  /usr/local/cuda-11.8  /usr/local/cuda-12  /usr/local/cuda-12.2
/usr/local/cuda/bin/nvcc


In [ ]:
import os
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-11.8/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-11.8/lib64:{ld}"
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
# sys.pathの設定を確認
# 次の設定は不要
# sys.path.append(f'{path_38}/lib/python3.8/site-packages')

In [ ]:
import sys
print(f'Python Version: {sys.version}')
for path in sys.path:
    print(path)

Python Version: 3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]
/content
/env/python
/usr/local_py38/lib/python38.zip
/usr/local_py38/lib/python3.8
/usr/local_py38/lib/python3.8/lib-dynload

/usr/local_py38/lib/python3.8/site-packages


In [ ]:
import os
path_org = os.environ['PATH']
path_38 = '/usr/local_py38'
path_38bin = f'{path_38}/bin'
os.environ['PATH'] = f'{path_38bin}:{path_org}'
# 設定確認
print(os.environ['PATH'])

/usr/local_py38/bin:/usr/local/cuda-11.8/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin


### add libraries

In [ ]:
!python --version

Python 3.8.8


In [ ]:
!ls -d /usr/local/cuda-*
!which nvcc

/usr/local/cuda-11  /usr/local/cuda-11.8  /usr/local/cuda-12  /usr/local/cuda-12.2
/usr/local/cuda-11.8/bin/nvcc


In [ ]:
!apt-get install python3-magic

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  python3-magic
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 12.6 kB of archives.
After this operation, 52.2 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 python3-magic all 2:0.4.24-2 [12.6 kB]
Fetched 12.6 kB in 0s (73.6 kB/s)
Selecting previously unselected package python3-magic.
(Reading database ... 129089 files and directories currently installed.)
Preparing to unpack .../python3-magic_2%3a0.4.24-2_all.deb ...
Unpacking python3-magic (2:0.4.24-2) ...
Setting up python3-magic (2:0.4.24-2) ...


In [ ]:
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio==0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
#!pip install torch==1.10.1+cu118 torchvision==0.11.2+cu118 torchaudio==0.10.1+cu118 -f https://download.pytorch.org/whl/cu118/torch_stable.html
# !pip install torchtext==0.11.1 pytorch-lightning==1.5.8 pytorch-metric-learning==1.1.0 timm==0.5.4 augly==0.2.1 addict==2.4.0 rich==11.0.0 faiss-cpu==1.7.2
!pip install torchtext==0.11.1 pytorch-lightning==1.5.8 pytorch-metric-learning==1.1.0 timm==0.5.4 addict==2.4.0 rich==11.0.0 faiss-cpu==1.7.2 #not insatll augly
!pip install -U scikit-learn
!pip install matplotlib
!pip install addict
!pip install albumentations

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |████████████████████████████████| 1821.4 MB 9.1 kB/s 
     |████████████████████████████████| 24.6 MB 1.2 MB/s 
     |████████████████████████████████| 2.9 MB 83.1 MB/s 
     |████████████████████████████████| 3.5 MB 16.3 MB/s 
     |████████████████████████████████| 8.0 MB 15.2 MB/s 
     |████████████████████████████████| 526 kB 77.5 MB/s 
     |████████████████████████████████| 106 kB 70.0 MB/s 
     |████████████████████████████████| 431 kB 66.8 MB/s 
     |████████████████████████████████| 215 kB 85.1 MB/s 
     |████████████████████████████████| 8.6 MB 81.2 MB/s 
     |████████████████████████████████| 168 kB 80.1 MB/s 
     |████████████████████████████████| 806 kB 80.5 MB/s 
     |████████████████████████████████| 840 kB 91.0 MB/s 
     |████████████████████████████████| 5.5 MB 76.6 MB/s 
     |████████████████████████████████| 736 kB 73.9 MB/s 
     |████████████████████████████████| 11.1 MB 76.0 

## コードの変更点

- Auglyは、比較用に消しておく
- 画像をcropするところも、データセットの画像がもともと切り抜き済だから削除していい

## train_convnext

In [ ]:
!nvidia-smi

Mon Dec 18 09:57:34 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
device = 'cuda'
import torch, gc
import os
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # Import

# In[ ]:


import os
import gc
import json
import yaml
import json
import copy
import pickle
import random
import shutil
import itertools
import typing as tp
from typing import Any, Dict, List, Optional
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
from addict import Dict as yaml2dic

from sklearn.model_selection import StratifiedKFold, GroupKFold, StratifiedGroupKFold

import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl
from pytorch_lightning import Trainer, LightningDataModule, LightningModule
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning.plugins import DDPPlugin
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, RichProgressBar, EarlyStopping, ProgressBar
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.loggers.csv_logs import CSVLogger

from pytorch_metric_learning import losses

import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations.core.transforms_interface import (
    DualTransform,
    ImageOnlyTransform,
    NoOp,
    to_tuple,
)

# colabでwidgetを使えるように
# https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb#scrollTo=w_63qoetcY8C
from google.colab import output
# output.enable_custom_widget_manager()
# output.disable_custom_widget_manager()

# from augly.image.functional import overlay_emoji, overlay_image, overlay_text
# from augly.image.transforms import BaseTransform
# from augly.utils import pathmgr
# from augly.utils.base_paths import MODULE_BASE_DIR
# from augly.utils.constants import FONT_LIST_PATH, FONTS_DIR, SMILEY_EMOJI_DIR

import timm
import faiss

import warnings
warnings.simplefilter('ignore')

# # Define Functions

# In[ ]:


def read_yaml(fpath):
    with open(fpath, mode="r") as file:
        yml = yaml.load(file, Loader=yaml.Loader)
        return yaml2dic(yml)


# ## Dataset

# In[ ]:


class JpoDataset_label(Dataset):
    def __init__(self, cfg, df, base_transform=None, hard_transform=None, mode=False):
        self.path = df["path"].values
        if "label" in df.keys():
            self.label = df["label"].values
        else:
            self.label = None
        if "cite_path" in df.keys():
            self.cite_path = df["cite_path"].values
        else:
            self.cite_path = None
        self._base_transform = base_transform
        self._hard_transform = hard_transform
        self.mode = mode
        self._cfg = cfg

    def __len__(self):
        return len(self.path)

    def __getitem__(self, idx):
        if (self.mode=='test')or(self.mode=='pred'):
            # test
            path = self.path[idx]
            if self.mode=='test':
                img = self.__read_image(SETTINGS['PROCESSED_DATA_DIR'] + f'crop_apply_images/{path}')
            else:
                img = self.__read_image(SETTINGS['PROCESSED_DATA_DIR'] + f'crop_cite_images/{path}')
            img = self._base_transform(image=img)['image']
            return {'images':img}
        elif self.mode=='val':
            # train or val
            path, cite_path = self.path[idx], self.cite_path[idx]
            label = self.label[idx]
            img = self.__read_image(SETTINGS['PROCESSED_DATA_DIR'] + f'crop_apply_images/{path}')
            cite_img = self.__read_image(SETTINGS['PROCESSED_DATA_DIR'] + f'crop_cite_images/{cite_path}')
            imgs = [self._base_transform(image=img)['image']] + [self._base_transform(image=cite_img)['image']]
            return {'images':imgs, 'labels':label}
        else:
            # train or val
            path, cite_path = self.path[idx], self.cite_path[idx]
            label = self.label[idx]
            img = self.__read_image(SETTINGS['PROCESSED_DATA_DIR'] + f'crop_apply_images/{path}')
            cite_img = self.__read_image(SETTINGS['PROCESSED_DATA_DIR'] + f'crop_cite_images/{cite_path}')
            imgs = [self._base_transform(image=img)['image']] + [self._base_transform(image=cite_img)['image']]
            for i in range(self._cfg.General.ncrops -2):
                if i%2 ==0:
                    imgs += [self._hard_transform(image=img)['image']]
                else:
                    imgs += [self._hard_transform(image=cite_img)['image']]

            return {'images':imgs, 'labels':label}


    def __read_image(self, path):
        img = Image.open(path)
        img = img.convert('RGB')
        img = np.array(img)
        # img = img.astype('float32')
        return img


# ## Augmentation

# In[ ]:


import pickle
import random
from typing import Any, Dict, List, Optional

# from augly.image.functional import overlay_emoji, overlay_image, overlay_text
# from augly.image.transforms import BaseTransform
# from augly.utils import pathmgr
# from augly.utils.base_paths import MODULE_BASE_DIR
# from augly.utils.constants import FONT_LIST_PATH, FONTS_DIR, SMILEY_EMOJI_DIR

from albumentations.core.transforms_interface import (
    DualTransform,
    ImageOnlyTransform,
    NoOp,
    to_tuple,
)

# class MakePadTextLogo(ImageOnlyTransform):
#     def __init__(
#         self,
#         opacity: float = 1.0,
#         size: int = 256,
#         p: float = 1.0,
#     ):
#         super().__init__(p)
#         self.opacity = opacity
#         self.size = size

#         with open(Path(FONTS_DIR) / FONT_LIST_PATH) as f:
#             font_list = [s.strip() for s in f.readlines()]
#             blacklist = [
#                 'TypeMyMusic',
#                 'PainttheSky-Regular',
#             ]
#             self.font_list = [
#                 f for f in font_list
#                 if all(_ not in f for _ in blacklist)
#             ]

#         self.font_lens = []
#         for ff in self.font_list:
#             font_file = Path(MODULE_BASE_DIR) / ff.replace('.ttf', '.pkl')
#             with open(font_file, 'rb') as f:
#                 self.font_lens.append(len(pickle.load(f)))

#     def apply(
#         self, image: np.array, metadata: Optional[List[Dict[str, Any]]] = None, **params
#     ) -> np.array:

#         base_image = image.copy()
#         h, w, _ = base_image.shape

#         back_img = np.zeros((self.size, self.size, 3), dtype=image.dtype)
#         back_img[:] = 255

#         i = random.randrange(0, len(self.font_list))
#         # i = random.choice([255, 62, 80, 220, 350, 157, 319, 52, 0])
#         kwargs = dict(
#             font_file=Path(MODULE_BASE_DIR) / self.font_list[i],
#             font_size=0,
#             # font_size=random.uniform(0.05, 0.2),
#             # color=[random.randrange(0, 256) for _ in range(3)],
#             color=[0,0,0],
#             x_pos=0,
#             # x_pos=random.uniform(0.0, 0.75),
#             metadata=metadata,
#             opacity=self.opacity,
#         )

#         mode = random.choice(['vertical', 'horizontal'])

#         if mode == 'vertical':
#             if np.random.rand() > 0.3:
#                 resize_rate = np.random.rand() * 0.3 + 0.5
#                 resize_h, resize_w = int(h*resize_rate), int(w*resize_rate)
#                 resize_img = Image.fromarray(base_image)
#                 resize_img = resize_img.resize((resize_w, resize_h)) # w,h

#                 start_w = int((self.size-resize_w)/2)
#                 use_img = Image.fromarray(back_img)

#                 text_num = random.randrange(1, 3)

#                 offset = int(random.uniform(0, 0.2)*self.size)
#                 img_bottom = (resize_h+offset)/self.size
#                 kwargs['font_size'] = (1-img_bottom)/text_num
#                 kwargs['font_size'] = np.min([kwargs['font_size'], 0.2])
#                 # kwargs['x_pos'] = random.uniform(0.1, 0.5)
#                 try:
#                     for j in range(text_num):
#                         if j == 0:
#                             y_pos = random.uniform(img_bottom, 1-(kwargs['font_size']*text_num))
#                         else:
#                             y_pos += kwargs['font_size']
#                         text = [random.randrange(0, self.font_lens[i]) for _ in range(random.randrange(5, 15))]
#                         kwargs['x_pos'] = np.max([0.5-(len(text)*0.3*kwargs['font_size']), 0])
#                         use_img = overlay_text(
#                             use_img,
#                             text=text,
#                             y_pos=y_pos,
#                             **kwargs,
#                         )
#                     use_img = np.array(use_img)
#                     use_img[offset:resize_h+offset, start_w:(start_w+resize_w)] = resize_img
#                     return use_img
#                 except OSError:
#                     return image
#             else:
#                 resize_rate = np.random.rand() * 0.3 + 0.3
#                 resize_h, resize_w = int(h*resize_rate), int(w*resize_rate)
#                 resize_img = Image.fromarray(base_image)
#                 resize_img = resize_img.resize((resize_w, resize_h)) # w,h

#                 start_w = int((self.size-resize_w)/2)
#                 use_img = Image.fromarray(back_img)

#                 text_num = random.randrange(1, 4)

#                 offset = int(random.uniform(0, 0.2)*self.size)
#                 img_up = (resize_h+offset)/self.size
#                 kwargs['font_size'] = (1-img_up)/text_num*0.8
#                 kwargs['font_size'] = np.min([kwargs['font_size'], 0.2])
#                 try:
#                     for j in range(text_num):
#                         if j == 0:
#                             y_pos = random.uniform(0, 1-img_up-(kwargs['font_size']*text_num))
#                         else:
#                             y_pos += kwargs['font_size']
#                         text = [random.randrange(0, self.font_lens[i]) for _ in range(random.randrange(5, 15))]
#                         kwargs['x_pos'] = np.max([0.5-(len(text)*0.3*kwargs['font_size']), 0])
#                         use_img = overlay_text(
#                             use_img,
#                             text=text,
#                             y_pos=y_pos,
#                             **kwargs,
#                         )
#                     use_img = np.array(use_img)
#                     if offset>0:
#                         use_img[-(resize_h+offset):-offset, start_w:(start_w+resize_w)] = resize_img
#                     else:
#                         use_img[-resize_h:, start_w:(start_w+resize_w)] = resize_img
#                     return use_img
#                 except OSError:
#                     return image
#         else:
#             if np.random.rand() > 0.5:
#                 resize_rate = np.random.rand() * 0.3 + 0.4
#                 resize_h, resize_w = int(h*resize_rate), int(w*resize_rate)
#                 resize_img = Image.fromarray(base_image)
#                 resize_img = resize_img.resize((resize_w, resize_h)) # w,h

#                 start_h = int((self.size-resize_h)/2)
#                 use_img = Image.fromarray(back_img)

#                 text_num = random.randrange(1, 3)

#                 img_right = resize_w/self.size
#                 kwargs['font_size'] = random.uniform(0.1, 0.2)
#                 # kwargs['x_pos'] = random.uniform(0.1, 0.5)
#                 try:
#                     for j in range(text_num):
#                         if j == 0:
#                             y_pos = random.uniform(0.3, 0.7-(kwargs['font_size']*text_num))
#                         else:
#                             y_pos += kwargs['font_size']
#                         text = [random.randrange(0, self.font_lens[i]) for _ in range(random.randrange(5, 15))]
#                         kwargs['x_pos'] = img_right+0.05
#                         use_img = overlay_text(
#                             use_img,
#                             text=text,
#                             y_pos=y_pos,
#                             **kwargs,
#                         )
#                     use_img = np.array(use_img)
#                     use_img[start_h:(start_h+resize_h), :resize_w] = resize_img
#                     return use_img
#                 except OSError:
#                     return image
#             else:
#                 resize_rate = np.random.rand() * 0.3 + 0.4
#                 resize_h, resize_w = int(h*resize_rate), int(w*resize_rate)
#                 resize_img = Image.fromarray(base_image)
#                 resize_img = resize_img.resize((resize_w, resize_h)) # w,h

#                 start_h = int((self.size-resize_h)/2)
#                 use_img = Image.fromarray(back_img)

#                 text_num = random.randrange(1, 3)

#                 img_right = resize_w/self.size
#                 kwargs['font_size'] = random.uniform(0.1, 0.2)
#                 # kwargs['x_pos'] = random.uniform(0.1, 0.5)
#                 try:
#                     for j in range(text_num):
#                         if j == 0:
#                             y_pos = random.uniform(0.3, 0.7-(kwargs['font_size']*text_num))
#                         else:
#                             y_pos += kwargs['font_size']
#                         text = [random.randrange(0, self.font_lens[i]) for _ in range(random.randrange(10, 15))]
#                         kwargs['x_pos'] = 0.02
#                         use_img = overlay_text(
#                             use_img,
#                             text=text,
#                             y_pos=y_pos,
#                             **kwargs,
#                         )
#                     use_img = np.array(use_img)
#                     use_img[start_h:(start_h+resize_h), -resize_w:] = resize_img
#                     return use_img
#                 except OSError:
#                     return image

#     def get_params(self):
#         return {"metadata": []}

#     def get_transform_init_args_names(self):
#         return ("overlaytext",)


# ## Lightning DataModule

# In[ ]:


class DataModule(LightningDataModule):
    def __init__(
        self,
        train_df,
        val_df,
        cfg,
        test_df=None
    ):
        super().__init__()
        self._train_df = train_df
        self._val_df = val_df
        self._cfg = cfg
        self._test_df = test_df

    def __create_dataset(self, mode=None):
        _dataset = eval(self._cfg.Dataset.type)
        if mode=='train':
            base_transform = self.__get_transform(self._cfg.Transform.train.base)
            hard_transform = self.__get_transform(self._cfg.Transform.train.hard)
            dataset = _dataset(self._cfg, self._train_df, base_transform, hard_transform, mode)
        elif mode=='val':
            transform = self.__get_transform(self._cfg.Transform.val)
            dataset = _dataset(self._cfg, self._val_df, transform, transform, mode)
        elif mode=='test':
            transform = self.__get_transform(self._cfg.Transform.test)
            dataset = _dataset(self._cfg, self._test_df, transform, None, mode)
        elif mode=='pred':
            transform = self.__get_transform(self._cfg.Transform.test)
            dataset = _dataset(self._cfg, self._test_df, transform, None, mode)
        return dataset

    def train_dataloader(self):
        dataset = self.__create_dataset(mode='train')
        return DataLoader(dataset, **self._cfg.Dataset.loader.train)

    def val_dataloader(self):
        dataset = self.__create_dataset(mode='val')
        return DataLoader(dataset, **self._cfg.Dataset.loader.val)

    def test_dataloader(self):
        dataset = self.__create_dataset(mode='test')
        return DataLoader(dataset, **self._cfg.Dataset.loader.test)

    def predict_dataloader(self):
        dataset = self.__create_dataset(mode='pred')
        return DataLoader(dataset, **self._cfg.Dataset.loader.test)

    #ref: https://github.com/kentaroy47/Kaggle-PANDA-1st-place-solution
    def __get_transform(self, conf_augmentation):
        def __get_object(trans):
            if trans.name in {"Compose", "OneOf"}:
                if str(trans.member)[:4] == 'list':
                    augs_tmp = eval(trans.member)
                else:
                    augs_tmp = [__get_object(aug) for aug in trans.member]
                return getattr(A, trans.name)(augs_tmp, **trans.params)

            if hasattr(A, trans.name):
                return getattr(A, trans.name)(**trans.params)
            else:
                return eval(trans.name)(**trans.params)

        if conf_augmentation is None:
            return None
        else:
            augs = [__get_object(aug) for aug in conf_augmentation]
            augs.append(ToTensorV2())
            return A.Compose(augs)


# ## Model

# In[ ]:


# Copyright (c) Meta Platforms, Inc. and affiliates.

# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.


import torch
import torch.nn as nn
import torch.nn.functional as F
from timm.models.layers import trunc_normal_, DropPath
from timm.models.registry import register_model

class Block(nn.Module):
    r""" ConvNeXt Block. There are two equivalent implementations:
    (1) DwConv -> LayerNorm (channels_first) -> 1x1 Conv -> GELU -> 1x1 Conv; all in (N, C, H, W)
    (2) DwConv -> Permute to (N, H, W, C); LayerNorm (channels_last) -> Linear -> GELU -> Linear; Permute back
    We use (2) as we find it slightly faster in PyTorch

    Args:
        dim (int): Number of input channels.
        drop_path (float): Stochastic depth rate. Default: 0.0
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
    """
    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim) # depthwise conv
        self.norm = LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, 4 * dim) # pointwise/1x1 convs, implemented with linear layers
        self.act = nn.GELU()
        self.pwconv2 = nn.Linear(4 * dim, dim)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones((dim)),
                                    requires_grad=True) if layer_scale_init_value > 0 else None
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.dwconv(x)
        x = x.permute(0, 2, 3, 1) # (N, C, H, W) -> (N, H, W, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma * x
        x = x.permute(0, 3, 1, 2) # (N, H, W, C) -> (N, C, H, W)

        x = input + self.drop_path(x)
        return x

class ConvNeXt(nn.Module):
    r""" ConvNeXt
        A PyTorch impl of : `A ConvNet for the 2020s`  -
          https://arxiv.org/pdf/2201.03545.pdf
    Args:
        in_chans (int): Number of input image channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1000
        depths (tuple(int)): Number of blocks at each stage. Default: [3, 3, 9, 3]
        dims (int): Feature dimension at each stage. Default: [96, 192, 384, 768]
        drop_path_rate (float): Stochastic depth rate. Default: 0.
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
        head_init_scale (float): Init scaling value for classifier weights and biases. Default: 1.
    """
    def __init__(self, in_chans=3, num_classes=1000,
                 depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], drop_path_rate=0.,
                 layer_scale_init_value=1e-6, head_init_scale=1.,
                 ):
        super().__init__()
        self.dims = dims

        self.downsample_layers = nn.ModuleList() # stem and 3 intermediate downsampling conv layers
        stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first")
        )
        self.downsample_layers.append(stem)
        for i in range(3):
            downsample_layer = nn.Sequential(
                    LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                    nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)

        self.stages = nn.ModuleList() # 4 feature resolution stages, each consisting of multiple residual blocks
        dp_rates=[x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[Block(dim=dims[i], drop_path=dp_rates[cur + j],
                layer_scale_init_value=layer_scale_init_value) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]

        self.norm = nn.LayerNorm(dims[-1], eps=1e-6) # final norm layer
        # self.head = nn.Linear(dims[-1], num_classes)
        self.head = nn.Identity()

        self.apply(self._init_weights)
        # self.head.weight.data.mul_(head_init_scale)
        # self.head.bias.data.mul_(head_init_scale)

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=.02)
            nn.init.constant_(m.bias, 0)

    def forward_features(self, x):
        for i in range(4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
        return self.norm(x.mean([-2, -1])) # global average pooling, (N, C, H, W) -> (N, C)

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x

class LayerNorm(nn.Module):
    r""" LayerNorm that supports two data formats: channels_last (default) or channels_first.
    The ordering of the dimensions in the inputs. channels_last corresponds to inputs with
    shape (batch_size, height, width, channels) while channels_first corresponds to inputs
    with shape (batch_size, channels, height, width).
    """
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError
        self.normalized_shape = (normalized_shape, )

    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x


model_urls = {
    "convnext_tiny_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth",
    "convnext_small_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_small_1k_224_ema.pth",
    "convnext_base_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_base_1k_224_ema.pth",
    "convnext_large_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_large_1k_224_ema.pth",
    "convnext_base_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_224.pth",
    "convnext_large_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_224.pth",
    "convnext_xlarge_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_xlarge_22k_224.pth",
    "convnext_base_22k_384": "https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_1k_384.pth"
}

@register_model
def convnext_tiny(pretrained=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], **kwargs)
    if pretrained:
        url = model_urls['convnext_tiny_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)
        model.load_state_dict(checkpoint["model"])
    return model

@register_model
def convnext_small(pretrained=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 27, 3], dims=[96, 192, 384, 768], **kwargs)
    if pretrained:
        url = model_urls['convnext_small_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)
        model.load_state_dict(checkpoint["model"])
    return model

@register_model
def convnext_base(pretrained=False, in_22k=False, img_size=224, **kwargs):
    num_classes = 21841 if in_22k else 1000
    model = ConvNeXt(depths=[3, 3, 27, 3], dims=[128, 256, 512, 1024], num_classes=num_classes,**kwargs)
    if pretrained:
        if img_size==224:
            url = model_urls['convnext_base_22k'] if in_22k else model_urls['convnext_base_1k']
        else:
            url = model_urls['convnext_base_22k_384']
        # print(f'load from : {url}')
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)
        model.load_state_dict(checkpoint["model"], strict=False)
    return model

@register_model
def convnext_large(pretrained=False, in_22k=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 27, 3], dims=[192, 384, 768, 1536], **kwargs)
    if pretrained:
        url = model_urls['convnext_large_22k'] if in_22k else model_urls['convnext_large_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)
        model.load_state_dict(checkpoint["model"])
    return model

@register_model
def convnext_xlarge(pretrained=False, in_22k=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 27, 3], dims=[256, 512, 1024, 2048], **kwargs)
    if pretrained:
        url = model_urls['convnext_xlarge_22k'] if in_22k else model_urls['convnext_xlarge_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)
        model.load_state_dict(checkpoint["model"])
    return model


# In[ ]:


# 参考：https://github.com/lyakaap/ISC21-Descriptor-Track-1st/blob/6e3a51be54e1aae8f41a1703bc75da7c143b5c53/exp/v83.py#L103
class ConvNextEmbeddingModel(nn.Module):

    def __init__(
        self, base_name: str, fc_dim: int, in_22k=False, img_size=224,
        pretrained=False, in_channels: int=3, fc_drop=0.0, drop_path_rate=0.0
    ):
        """Initialize"""
        self.base_name = base_name
        super(ConvNextEmbeddingModel, self).__init__()

        # # prepare backbone
        base_model = eval(base_name)(pretrained=pretrained,
                                     drop_path_rate=drop_path_rate,
                                     in_22k=in_22k,
                                     img_size=img_size)
        in_features = base_model.dims[-1]
        self.backbone = base_model
        print(f"{base_name}: {in_features}")

        self.fc = nn.Linear(in_features, fc_dim, bias=False)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, x):
        batch_size = x.shape[0]
        x = self.backbone.forward_features(x)
        x = self.fc(x)
        # x = self.bn(x)
        x = F.normalize(x)
        return x


# ## Lightning Module

# In[ ]:


class Model(LightningModule):

    def __init__(self, cfg):
        super().__init__()
        self._cfg = cfg
        self._criterion = eval(self._cfg.Loss.name)(**self._cfg.Loss.params)                             if self._cfg.Loss.params                             else eval(self._cfg.Loss.name)()
        if self._cfg.CrossBatchMemory.enable:
            self._criterion = losses.CrossBatchMemory(self._criterion, **self._cfg.CrossBatchMemory.params)
        self.model = eval(self._cfg.Model.type)(**self._cfg.Model.params)
        self.log_metric = []


    def forward(self, x):
        out = self.model(x)
        return out


    def training_step(self, batch, batch_idx):
        loss, preds, labels = self.__share_step(batch, 'train')
        return {
            "loss": loss, "labels": labels, "preds": preds
        }

    def validation_step(self, batch, batch_idx):
        loss, preds, labels = self.__share_step(batch, 'val')
        return {
            "loss": loss, "labels": labels, "preds": preds
        }

    def predict_step(self, batch, batch_idx):
        feats = batch['features']
        preds = self.forward(feats).squeeze(1)
        return {
            "preds": preds.detach().cpu()
        }

    def __share_step(self, batch, mode):
        images, labels = batch['images'], batch['labels']
        images = torch.cat([image for image in images], dim=0)
        if mode=='train':
            labels = torch.tile(labels, dims=(self._cfg.General.ncrops,))
        else:
            labels = torch.tile(labels, dims=(2,))

        if self._cfg.General.debug:
            for i in range(10):
                plot_image = images[labels==labels[i]]
                plt.figure(figsize=(10,10))
                for i, img in enumerate(plot_image):
                    plt.subplot(3,3,i+1)
                    plt.imshow(img.detach().cpu().numpy().transpose(1,2,0))
                plt.show()

        logits = self.forward(images)
        loss = self._criterion(logits, labels)

        dic = dict()
        dic[f"{mode}_loss/fold_{self._cfg.General.fold}"] = loss.detach().item()
        self.log_dict(dic, prog_bar=True)

        return loss, logits.detach().cpu(), labels.detach().cpu()


    def training_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, 'train')

    def validation_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, 'val')

    def __share_epoch_end(self, outputs, mode):
        preds = []
        labels = []
        for out in outputs:
            pred, label = out['preds'], out['labels']
            preds.append(pred)
            labels.append(label)
        preds = torch.cat(preds)
        labels = torch.cat(labels)

        dic = dict()
        dic["epoch"] = self.current_epoch
        dic[f"{mode}_loss/fold_{self._cfg.General.fold}"] = torch.stack([o["loss"] for o in outputs]).detach().mean().item()
        if mode=='val':
            self.log_metric.append(dic[_cfg.Callbacks.ModelCheckpoint.monitor])
            if _cfg.Callbacks.ModelCheckpoint.mode == 'min':
                dic[f"best_{_cfg.Callbacks.ModelCheckpoint.monitor}"] = np.min(self.log_metric)
            else:
                dic[f"best_{_cfg.Callbacks.ModelCheckpoint.monitor}"] = np.max(self.log_metric)
        self.log_dict(dic, prog_bar=True)


    def configure_optimizers(self):
        optimizer = eval(self._cfg.Optimizer.name)(self.parameters(), **self._cfg.Optimizer.params)
        scheduler = {'scheduler': eval(self._cfg.Scheduler.name)(optimizer, **self._cfg.Scheduler.params),
                    'interval': self._cfg.Scheduler.interval
                    }

        return [optimizer], [scheduler]

    def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx,
                   optimizer_closure, on_tpu, using_native_amp, using_lbfgs):
        optimizer.step(closure=optimizer_closure)


# ## inference

# In[ ]:


def to_device(
    tensors: tp.Union[tp.Tuple[torch.Tensor], tp.Dict[str, torch.Tensor]],
    device: torch.device, *args, **kwargs
):
    if isinstance(tensors, tuple):
        return (t.to(device, *args, **kwargs) for t in tensors)
    elif isinstance(tensors, dict):
        return {
            k: t.to(device, *args, **kwargs) for k, t in tensors.items()}
    else:
        return tensors.to(device, *args, **kwargs)


# In[ ]:


def run_inference_loop(model, loader, device):
    model.to(device)
    model.eval()
    pred_list = []
    with torch.no_grad():
        for batch in tqdm(loader):
            batch = to_device(batch, device)
            x = batch["images"]
            y = model(x)
            pred_list.append(y.detach().cpu().numpy())
            # pred_list.append(y.sigmoid().detach().cpu().numpy())

    pred_arr = np.concatenate(pred_list)
    del pred_list
    return pred_arr


# In[ ]:


class FaissKNeighbors:
    def __init__(self, k=20):
        self.index = None
        self.d = None
        self.k = k

    def fit(self, X):
        X = X.copy(order='C')
        self.d = X.shape[1]
        self.index = faiss.IndexFlatIP(self.d)
        # self.index = faiss.IndexFlatL2(self.d)
        self.index.add(X.astype(np.float32))

    def predict(self, X):
        X = X.copy(order='C')
        X = np.reshape(X, (-1, self.d))
        distances, indices = self.index.search(X.astype(np.float32), k=self.k)
        if X.shape[0] == 1:
            return distances[0], indices[0]
        else:
            return distances, indices


# # Read Config

# In[ ]:


CFG = read_yaml("./config/config_train_convnext.yaml")

SETTINGS = json.load(open('settings.json', 'r'))
SETTINGS['MODEL_CHECKPOINT_DIR'] += "convnext/"
SETTINGS['LOGS_DIR'] += "convnext/"


# # Read Data, Split folds

# In[ ]:


df_train = pd.read_csv(SETTINGS['PROCESSED_DATA_DIR'] + "train_w_group_label.csv")
df_test = pd.read_csv(SETTINGS['RAW_DATA_DIR'] + "test.csv")
df_cite = pd.read_csv(SETTINGS['RAW_DATA_DIR'] + "cite.csv")
smpl_sub = pd.read_csv(SETTINGS['RAW_DATA_DIR'] + "sample_submission.csv")


# ## Make fold

# In[ ]:


train = df_train.copy()
test = df_test.copy()


# In[ ]:


if CFG.General.num_folds>1:
    CV = StratifiedKFold(n_splits=CFG.General.num_folds, shuffle=True, random_state=CFG.General.seed)
    train["fold"] = -1
    for fold_id, (tr_idx, val_idx) in enumerate(CV.split(train["gid"], train["category"], train["gid"])):
        # print(val_idx)
        train.loc[val_idx, "fold"] = fold_id
else:
    train["fold"] = 0


# # Train

# In[ ]:

print('===== Train Start =====')

torch.cuda.empty_cache()
torch.cuda.memory_allocated()

for fold in range(CFG.General.num_folds):

    print(f"[fold {fold}]")
    _cfg = copy.deepcopy(CFG)
    seed_everything(_cfg.General.seed)

    _cfg.General.fold = fold
    _cfg.Callbacks.ModelCheckpoint.filename = f'best_{_cfg.Callbacks.ModelCheckpoint.monitor}_fold_{_cfg.General.fold}'
    _cfg.Callbacks.ModelCheckpoint.dirpath = SETTINGS['MODEL_CHECKPOINT_DIR']
    _cfg.Callbacks.ModelCheckpoint.monitor += f'/fold_{_cfg.General.fold}'
    if 'EarlyStopping' in _cfg.Callbacks.keys():
        _cfg.Callbacks.EarlyStopping.monitor += f'/fold_{_cfg.General.fold}'

    train_df = train[train["fold"] != fold]
    val_df = train[train["fold"] == fold]

    datamodule = DataModule(train_df, val_df, _cfg)
    model = Model(_cfg)

    loggers = []
    loggers.append(CSVLogger(save_dir=SETTINGS['LOGS_DIR'], name=f'fold_{fold}'))

    callbacks = []
    callbacks.append(RichProgressBar())
    callbacks.append(LearningRateMonitor())
    checkpoint_callback = ModelCheckpoint(**_cfg.Callbacks.ModelCheckpoint)
    callbacks.append(checkpoint_callback)
    if 'EarlyStopping' in _cfg.Callbacks.keys():
        callbacks.append(EarlyStopping(**_cfg.Callbacks.EarlyStopping))

    trainer = Trainer(
        max_epochs=_cfg.General.epoch,
        callbacks=callbacks,
        logger=loggers,
        default_root_dir=SETTINGS['LOGS_DIR'],
        fast_dev_run=_cfg.General.debug,
        # plugins=DDPPlugin(find_unused_parameters=False),
        **_cfg.Trainer
    )

    trainer.fit(model, datamodule=datamodule)

    shutil.copy(SETTINGS['MODEL_CHECKPOINT_DIR']+'last.ckpt', SETTINGS['MODEL_CHECKPOINT_DIR']+f'last_fold_{fold}.ckpt')

    # break


# # Inference OOF & Test

# In[ ]:


test_idx2id = dict(zip(df_test.index, df_test['gid']))
cite_idx2id = dict(zip(df_cite.index, df_cite['gid']))


# In[ ]:


tmp = train.copy()
tmp['path'] = train['cite_path']

train_embeds = []
val_embeds = []

print('===== Make OOF Embeddings =====')

for fold in range(CFG.General.num_folds):

    print(f"[fold {fold}]")

    val_idx = train.query("fold == @fold").index.values

    best_model_path = SETTINGS['MODEL_CHECKPOINT_DIR'] + f'last_fold_{fold}.ckpt'
    model = Model.load_from_checkpoint(best_model_path, cfg=CFG)

    # # inference
    datamodule = DataModule(None, None, CFG, train.loc[val_idx])
    val_embed = run_inference_loop(model, datamodule.test_dataloader(), CFG.General.device)

    datamodule = DataModule(None, None, CFG, tmp)
    train_embed = run_inference_loop(model, datamodule.predict_dataloader(), CFG.General.device)

    val_embeds.append(val_embed)
    train_embeds.append(train_embed)
    # break


# In[ ]:


print('===== OOF Score =====')

for fold in range(CFG.General.num_folds):
    print(f"[fold {fold}]")
    val_idx = train.query("fold == @fold").index.values

    kn = FaissKNeighbors()
    kn.fit(train_embeds[fold])
    dist, idx = kn.predict(val_embeds[fold])

    cnt = 0
    for n, idx_ in tqdm(enumerate(idx)):
        cites = [train['cite_gid'].iloc[i] for i in idx_]
        if train['cite_gid'].loc[val_idx].iloc[n] in cites:
            cnt += 1
    score = np.round(cnt/len(val_idx), 3)
    print(f'score = {score}, num = {cnt}')
    print()


# In[ ]:


cite_embeds = []
train_embeds = []

print('===== Make train/cite Embeddings =====')
for fold in range(CFG.General.num_folds):
    print(f"[fold {fold}]")

    best_model_path = SETTINGS['MODEL_CHECKPOINT_DIR'] + f'last_fold_{fold}.ckpt'
    model = Model.load_from_checkpoint(best_model_path, cfg=CFG)

    # # inference
    datamodule = DataModule(None, None, CFG, df_cite)
    cite_embed = run_inference_loop(model, datamodule.predict_dataloader(), CFG.General.device)

    datamodule = DataModule(None, None, CFG, df_train)
    train_embed = run_inference_loop(model, datamodule.test_dataloader(), CFG.General.device)

    if (np.isnan(np.sum(cite_embed)) == False) & (np.isnan(np.sum(train_embed)) == False):
        print(f'fold {fold} Embedding OK')
    else:
        print(f'fold {fold} Embedding contain Nan. Please Retry.')

    np.save(SETTINGS['MODEL_CHECKPOINT_DIR'] + f'/cite_embed_{fold}.npy', cite_embed)
    np.save(SETTINGS['MODEL_CHECKPOINT_DIR'] + f'/train_embed_{fold}.npy', train_embed)

    cite_embeds.append(cite_embed)
    train_embeds.append(train_embed)


# In[ ]:


cite_mean_embed = np.mean(cite_embeds, axis=0)
train_mean_embed = np.mean(train_embeds, axis=0)


# In[ ]:


print('===== Save train/cite Embeddings =====')
np.save(SETTINGS['MODEL_CHECKPOINT_DIR'] + '/cite_mean_embed.npy', cite_mean_embed)
np.save(SETTINGS['MODEL_CHECKPOINT_DIR'] + '/train_mean_embed.npy', train_mean_embed)


# In[ ]:


print('===== Finish =====')

In [ ]:
# save yolov8 ver output to your env
!mkdir "your_colab_local_path/yolov8/models"
!mkdir "your_colab_local_path/yolov8/logs"

!cp -r /content/models/convnext/* "your_colab_local_path/yolov8/models"
!cp -r /content/logs/convnext/* "your_colab_local_path/yolov8/logs"